In [3]:
import numpy as np
import pandas as pd
import copy
import ast

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


<h1 align='center'>Data Preporccessing<h1>

In [4]:
df1 = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
df2 = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
print(df1.shape)
print(df2.shape)

(4803, 20)
(4803, 4)


In [5]:
df2['id'] = df2['movie_id']
df = df1.merge(df2, on='id')
print(df.shape)

(4803, 24)


In [11]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
movies = df[['id','title_x','overview','genres',
             'keywords','original_language','production_companies',
             'tagline','cast','crew']]
movies['overview'] = movies['overview'].fillna(" ")
movies['tagline'] = movies['tagline'].fillna(" ")
movies.head(1)

/tmp/ipykernel_37/1364679370.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['overview'] = movies['overview'].fillna(" ")
/tmp/ipykernel_37/1364679370.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tagline'] = movies['tagline'].fillna(" ")


,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


# Feature engeneering

In [9]:
# first make a copy
movies2 = movies.copy()

In [12]:
# function fr genres column
def convert_genres(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(f"gen_{i['name']}")
    text = ", ".join(List)
    text = text.lower().replace(" ","").replace(","," ")
    return text

# function fr keywords column
def convert_keywords(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    text = ", ".join(List)
    text = text.lower().replace(" ","").replace(",", " ")
    return text

# function for productin companies
def convert_pc(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    text = ", ".join(List)
    text = text.lower().replace(" ","").replace(","," ").replace("."," ").replace("  "," ")
    return text

# funciton for extracting top 5 cast
def top_cast(obj):
    data = ast.literal_eval(obj)
    top5 = data[:5]
    top5_cast = [] 
    for member in top5:
        name = member['name'].replace(' ','').replace('"'," ").replace("."," ").replace("-","").lower()
        character = member['character'].replace(' ','').replace('"'," ").replace("."," ").replace("-","").lower()
        top5_cast.append(f"{name} {character}")
    return " ".join(top5_cast) 

# for extrating crew (director)
def fetch_director(obj):
    data = ast.literal_eval(obj)
    director = []
    for i in data:
        if i['job'] == 'Director':
            name = i['name'].replace(' ','').replace('"'," ").replace("."," ").replace("-","").lower()
            director.append(f"{name}")
    return " ".join(director)

# applying function 
movies2['genres'] = movies2['genres'].apply(convert_genres)
movies2['keywords'] = movies2['keywords'].apply(convert_keywords)
movies2['production_companies'] = movies2['production_companies'].apply(convert_pc)
movies2['cast'] = movies2['cast'].apply(top_cast)
movies2['crew'] = movies2['crew'].apply(fetch_director)

# converting original language
movies2['original_language'] = "ol_" + movies2['original_language'].astype(str)
movies2.head(1)

In [41]:
movies7['title'] = movies7['title_x']

In [42]:
columns = ['title_x','overview','genres','keywords',
           'original_language','production_companies',
           'tagline','cast','crew']
for i in columns:
    movies7[i] = movies7[i].apply(lambda x:x.lower().split())

In [286]:
movies7.head(2)

,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew,title
0,19995,[avatar],"[in, the, 22nd, century,, a, paraplegic, marin...","[gen_action, gen_adventure, gen_fantasy, gen_s...","[cultureclash, future, spacewar, spacecolony, ...",[ol_en],"[ingeniousfilmpartners, twentiethcenturyfoxfil...","[enter, the, world, of, pandora.]","[samworthington, jakesully, zoesaldana, neytir...",[jamescameron],Avatar
1,285,"[pirates, of, the, caribbean:, at, world's, end]","[captain, barbossa,, long, believed, to, be, d...","[gen_adventure, gen_fantasy, gen_action]","[ocean, drugabuse, exoticisland, eastindiatrad...",[ol_en],"[waltdisneypictures, jerrybruckheimerfilms, se...","[at, the, end, of, the, world,, the, adventure...","[johnnydepp, captainjacksparrow, orlandobloom,...",[goreverbinski],Pirates of the Caribbean: At World's End


## Conatinating all Columns into `tags`

In [43]:
movies8 = movies7.copy()

In [44]:
movies8["tags"] = movies8['title_x'] + movies8['overview'] + movies8['genres'] + movies8['keywords'] + movies8['original_language'] + movies8['production_companies'] + movies8['tagline'] + movies8['cast'] + movies8['crew']

In [290]:
movies8.head()

,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew,title,tags
0,19995,[avatar],"[in, the, 22nd, century,, a, paraplegic, marin...","[gen_action, gen_adventure, gen_fantasy, gen_s...","[cultureclash, future, spacewar, spacecolony, ...",[ol_en],"[ingeniousfilmpartners, twentiethcenturyfoxfil...","[enter, the, world, of, pandora.]","[samworthington, jakesully, zoesaldana, neytir...",[jamescameron],Avatar,"[avatar, in, the, 22nd, century,, a, paraplegi..."
1,285,"[pirates, of, the, caribbean:, at, world's, end]","[captain, barbossa,, long, believed, to, be, d...","[gen_adventure, gen_fantasy, gen_action]","[ocean, drugabuse, exoticisland, eastindiatrad...",[ol_en],"[waltdisneypictures, jerrybruckheimerfilms, se...","[at, the, end, of, the, world,, the, adventure...","[johnnydepp, captainjacksparrow, orlandobloom,...",[goreverbinski],Pirates of the Caribbean: At World's End,"[pirates, of, the, caribbean:, at, world's, en..."
2,206647,[spectre],"[a, cryptic, message, from, bond’s, past, send...","[gen_action, gen_adventure, gen_crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",[ol_en],"[columbiapictures, danjaq, b24]","[a, plan, no, one, escapes]","[danielcraig, jamesbond, christophwaltz, blofe...",[sammendes],Spectre,"[spectre, a, cryptic, message, from, bond’s, p..."
3,49026,"[the, dark, knight, rises]","[following, the, death, of, district, attorney...","[gen_action, gen_crime, gen_drama, gen_thriller]","[dccomics, crimefighter, terrorist, secretiden...",[ol_en],"[legendarypictures, warnerbros, dcentertainmen...","[the, legend, ends]","[christianbale, brucewayne/batman, michaelcain...",[christophernolan],The Dark Knight Rises,"[the, dark, knight, rises, following, the, dea..."
4,49529,"[john, carter]","[john, carter, is, a, war-weary,, former, mili...","[gen_action, gen_adventure, gen_sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...",[ol_en],[waltdisneypictures],"[lost, in, our, world,, found, in, another.]","[taylorkitsch, johncarter, lynncollins, dejaht...",[andrewstanton],John Carter,"[john, carter, john, carter, is, a, war-weary,..."


In [45]:
movies9 = pd.DataFrame(movies8[['id', 'title', 'tags']])

In [293]:
movies9.head()

,id,title,tags
0,19995,Avatar,"[avatar, in, the, 22nd, century,, a, paraplegi..."
1,285,Pirates of the Caribbean: At World's End,"[pirates, of, the, caribbean:, at, world's, en..."
2,206647,Spectre,"[spectre, a, cryptic, message, from, bond’s, p..."
3,49026,The Dark Knight Rises,"[the, dark, knight, rises, following, the, dea..."
4,49529,John Carter,"[john, carter, john, carter, is, a, war-weary,..."


In [294]:
movies9.iloc[6].tags


['tangled',
 'when',
 'the',
 "kingdom's",
 'most',
 'wanted-and',
 'most',
 'charming-bandit',
 'flynn',
 'rider',
 'hides',
 'out',
 'in',
 'a',
 'mysterious',
 'tower,',
 "he's",
 'taken',
 'hostage',
 'by',
 'rapunzel,',
 'a',
 'beautiful',
 'and',
 'feisty',
 'tower-bound',
 'teen',
 'with',
 '70',
 'feet',
 'of',
 'magical,',
 'golden',
 'hair.',
 "flynn's",
 'curious',
 'captor,',
 "who's",
 'looking',
 'for',
 'her',
 'ticket',
 'out',
 'of',
 'the',
 'tower',
 'where',
 "she's",
 'been',
 'locked',
 'away',
 'for',
 'years,',
 'strikes',
 'a',
 'deal',
 'with',
 'the',
 'handsome',
 'thief',
 'and',
 'the',
 'unlikely',
 'duo',
 'sets',
 'off',
 'on',
 'an',
 'action-packed',
 'escapade,',
 'complete',
 'with',
 'a',
 'super-cop',
 'horse,',
 'an',
 'over-protective',
 'chameleon',
 'and',
 'a',
 'gruff',
 'gang',
 'of',
 'pub',
 'thugs.',
 'gen_animation',
 'gen_family',
 'hostage',
 'magic',
 'horse',
 'fairytale',
 'musical',
 'princess',
 'animation',
 'tower',
 'blondewom

## Converting `tags` into string

In [46]:
movies9['tags'] = movies9['tags'].apply(lambda x: ' '.join(x))

In [53]:
movies9.iloc[6].tags

"tangled when the kingdom's most wantedand most charmingbandit flynn rider hides out in a mysterious tower he's taken hostage by rapunzel a beautiful and feisty towerbound teen with 70 feet of magical golden hair flynn's curious captor who's looking for her ticket out of the tower where she's been locked away for years strikes a deal with the handsome thief and the unlikely duo sets off on an actionpacked escapade complete with a supercop horse an overprotective chameleon and a gruff gang of pub thugs gen_animation gen_family hostage magic horse fairytale musical princess animation tower blondewoman selfishness healingpower basedonfairytale duringcreditsstinger healinggift animalsidekick ol_en waltdisneypictures waltdisneyanimationstudios they're taking adventure to new lengths zacharylevi flynnridervoice mandymoore rapunzelvoice donnamurphy mothergothelvoice ronperlman stabbingtonbrothervoice m c gainey captainoftheguardvoice byronhoward nathangreno"

#### Notice
we can see thre are still some symbols like `(voice)` `.` now we have to handle those

In [51]:
def clean_text(text):
    for ch in ['(', ')', '.', ',','-']:     # remove these characters completely
        text = text.replace(ch, '')
    text = ' '.join(text.split())       # replace multiple spaces with single space because it joinning only words with single space
    return text

In [52]:
movies9['tags'] = movies9['tags'].apply(clean_text)

In [301]:
movies9.head()

,id,title,tags
0,19995,Avatar,"avatar in the 22nd century, a paraplegic marin..."
1,285,Pirates of the Caribbean: At World's End,pirates of the caribbean: at world's end capta...
2,206647,Spectre,spectre a cryptic message from bond’s past sen...
3,49026,The Dark Knight Rises,the dark knight rises following the death of d...
4,49529,John Carter,"john carter john carter is a war-weary, former..."


## Handling simmiler words like `loved, loving` to `love, love` 

it will require `nltk` library from natural language proccessing. if you are running this notbook on your local computer, you may intsll it by `!pip install nltk`

In [56]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [63]:
def stem(text):
    a = []
    for i in text.split():
        a.append(ps.stem(i))
    return " ".join(a)

In [65]:
# just test of `stem`
print("tangled when the kingdom's most wantedand most charmingbandit flynn rider hides out in a mysterious tower he's taken hostage by rapunzel a beautiful and feisty towerbound teen with 70 feet of magical golden hair flynn's curious captor who's looking for her ticket out of the tower where she's been locked away for years strikes a deal with the handsome thief and the unlikely duo sets off on an actionpacked escapade complete with a supercop horse an overprotective chameleon and a gruff gang of pub thugs gen_animation gen_family hostage magic horse fairytale musical princess animation tower blondewoman selfishness healingpower basedonfairytale duringcreditsstinger healinggift animalsidekick ol_en waltdisneypictures waltdisneyanimationstudios they're taking adventure to new lengths zacharylevi flynnridervoice mandymoore rapunzelvoice donnamurphy mothergothelvoice ronperlman stabbingtonbrothervoice m c gainey captainoftheguardvoice byronhoward nathangreno\n"
)
stem("tangled when the kingdom's most wantedand most charmingbandit flynn rider hides out in a mysterious tower he's taken hostage by rapunzel a beautiful and feisty towerbound teen with 70 feet of magical golden hair flynn's curious captor who's looking for her ticket out of the tower where she's been locked away for years strikes a deal with the handsome thief and the unlikely duo sets off on an actionpacked escapade complete with a supercop horse an overprotective chameleon and a gruff gang of pub thugs gen_animation gen_family hostage magic horse fairytale musical princess animation tower blondewoman selfishness healingpower basedonfairytale duringcreditsstinger healinggift animalsidekick ol_en waltdisneypictures waltdisneyanimationstudios they're taking adventure to new lengths zacharylevi flynnridervoice mandymoore rapunzelvoice donnamurphy mothergothelvoice ronperlman stabbingtonbrothervoice m c gainey captainoftheguardvoice byronhoward nathangreno"
)

tangled when the kingdom's most wantedand most charmingbandit flynn rider hides out in a mysterious tower he's taken hostage by rapunzel a beautiful and feisty towerbound teen with 70 feet of magical golden hair flynn's curious captor who's looking for her ticket out of the tower where she's been locked away for years strikes a deal with the handsome thief and the unlikely duo sets off on an actionpacked escapade complete with a supercop horse an overprotective chameleon and a gruff gang of pub thugs gen_animation gen_family hostage magic horse fairytale musical princess animation tower blondewoman selfishness healingpower basedonfairytale duringcreditsstinger healinggift animalsidekick ol_en waltdisneypictures waltdisneyanimationstudios they're taking adventure to new lengths zacharylevi flynnridervoice mandymoore rapunzelvoice donnamurphy mothergothelvoice ronperlman stabbingtonbrothervoice m c gainey captainoftheguardvoice byronhoward nathangreno



"tangl when the kingdom' most wantedand most charmingbandit flynn rider hide out in a mysteri tower he' taken hostag by rapunzel a beauti and feisti towerbound teen with 70 feet of magic golden hair flynn' curiou captor who' look for her ticket out of the tower where she' been lock away for year strike a deal with the handsom thief and the unlik duo set off on an actionpack escapad complet with a supercop hors an overprotect chameleon and a gruff gang of pub thug gen_anim gen_famili hostag magic hors fairytal music princess anim tower blondewoman selfish healingpow basedonfairytal duringcreditssting healinggift animalsidekick ol_en waltdisneypictur waltdisneyanimationstudio they'r take adventur to new length zacharylevi flynnridervoic mandymoor rapunzelvoic donnamurphi mothergothelvoic ronperlman stabbingtonbrothervoic m c gainey captainoftheguardvoic byronhoward nathangreno"

#### now applyting `stem` funcitin to our current dataset's tags column

In [66]:
movies_stem = movies9.copy()

In [69]:
movies_stem['tags'] = movies_stem['tags'].apply(stem)

# Vectorizing

In [70]:
movies_stem.shape

(4803, 3)

In [78]:
vector = movies_stem.copy()

In [79]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [80]:
# covert into vector
vector = cv.fit_transform(movies_stem['tags']).toarray()

In [82]:
print(vector.shape)

(4803, 57809)


In [83]:
# check first few words
feature_name =cv.get_feature_names_out()
print(feature_name[:500])

['00' '000' '007' '05' '06' '07am' '10' '100' '1000' '10000' '100000'
 '1000000' '1000foot' '1001' '100bare' '10191' '1019entertain'
 '101ststreetfilm' '101yearold' '102' '108yearold' '10round' '10th'
 '10thholeproduct' '10weststudio' '10year' '10yearold' '10yearoldzo' '11'
 '11000' '117' '118' '119' '11th' '11year' '11yearold' '12' '1200'
 '120dbfilm' '120film' '121208' '1214' '1215' '1250' '125th' '127'
 '12foot' '12hour' '12th' '12thcenturi' '12year' '12yearold' '13' '130'
 '1300' '1350' '13ghostsproductionscanadainc' '13th' '13year' '13yearold'
 '14' '140' '14000' '1408' '141' '142' '1429' '148000' '1492pictur'
 '14foot' '14pm' '14th' '14year' '14yearold' '15' '150' '150th' '1520'
 '1536' '15mile' '15th' '15thcenturi' '15year' '15yearold' '16' '1600'
 '161' '1630s' '16441911' '1681' '1691' '16blockproduct' '16th'
 '16thcenturi' '16yearold' '17' '1700' '170000' '1718' '173rd' '1748'
 '1776' '17th' '17thcenturi' '17year' '17yearold' '18' '180' '1800' '1812'
 '1812product' '1818' '182

# Making `cosine distance`

In [85]:
from sklearn.metrics.pairwise import cosine_similarity

In [88]:
similarity = cosine_similarity(vector)

In [87]:
cosine_similarity(vector).shape

(4803, 4803)

In [91]:
similarity[:3]

array([[1.        , 0.08695652, 0.06552976, ..., 0.05285626, 0.02675241,
        0.01428717],
       [0.08695652, 1.        , 0.04914732, ..., 0.05285626, 0.04012862,
        0.02857434],
       [0.06552976, 0.04914732, 1.        , ..., 0.02987405, 0.01512031,
        0.01615005]])

now as we got similarity, we have to sort it in descending way. sothat we can get the top 5 or 10 similar movies according to the similarity value. but the problem is if we sort it, their index value will change. so we have to apply `enumerate` so that each sorted value keeps their orignal index position. 

there are two different `id` and `index`. id is for fetching the thumbnail and other things from TMDB site. they are usefull. but we will not use it right now. we will work on `index` value

In [130]:
# testing
print("ID: ", movies9[movies9['title'] == 'Batman Begins']['id'].iloc[0])
print("Index: ", movies9[movies9['title'] == 'Batman Begins'].index[0])

ID:  272
Index:  119


`enumerate` creats an object for each value : `enumerate(similarity[0])` we will put it into a list : `list(enumerate(similarity[0]))`.
here inside `[]` we can provide index position of any movie. and can see it's similarities with other movies. this is a `list` of `tupples`

In [ ]:
list(enumerate(similarity[1]))

#### 1:
now we can sort it. like: `sorted(list(enumerate(similarity[1])),reverse=True)` but it will show the last movie first. because we used reverse whitch thought we are sorting based on index. so we need to specify the sorting position.
current output:
```
[(4802, 0.028574344469642157),
 (4801, 0.0401286176952564),
...
```
#### 2:
if we think it as a csv, here first column `4882` is the last number of the index. so can tell that it sorted by first column `[0]` but we need to sort by column 2 `[1]`. to specify we will use a `lambda` function to give to set 2nd column as key. `sorted(list(enumerate(similarity[1])),reverse=True, key = lambda x:x[1])`

#### 3:
but it still have a problem. it show the relation with it also. which index is `[0]` but we need 2nd index `[1]`. and we will take first 10 similar movies, which index will be `[11]`. so we will put a positio `[1:11]`

In [142]:
# sorted(list(enumerate(similarity[1])),reverse=True)
# sorted(list(enumerate(similarity[0])),reverse=True, key = lambda x:x[1])
sorted(list(enumerate(similarity[0])),reverse=True, key = lambda x:x[1])[1:11]

[(2403, 0.2153527608232662),
 (1213, 0.20348923188911994),
 (778, 0.19332886183313755),
 (1086, 0.1930468356263361),
 (1914, 0.19206937701335358),
 (507, 0.18725147156828453),
 (539, 0.18363965490589892),
 (1201, 0.17518714874213512),
 (47, 0.16447838793172298),
 (3158, 0.15890330854706755)]

## Final function

now lets build the final functin that will do all the steps

In [155]:
def recommend(movie):
    movie_index = movies9[movies9['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True, key = lambda x:x[1])[1:11] # we can remove [0] as we did it n `distance`

    for i in movies_list:
        print(movies9.iloc[i[0]].title)

In [156]:
recommend('Avatar')

Aliens
Aliens vs Predator: Requiem
Meet Dave
Aliens in the Attic
Lifeforce
Independence Day
Titan A.E.
Predators
Star Trek Into Darkness
Alien


In [ ]:
import pickle
pickle.dump()

In [2]:
movies9.head()

NameError: name 'movies9' is not defined